In [1]:
#| include: false

import duckdb
import pandas as pd
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

%sql CREATE OR REPLACE TABLE user_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'user_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE courier_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'courier_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv('00_data/sql/orders.csv', header=True, columns={'order_id': 'INT', 'creation_time': 'TIMESTAMP', 'product_ids': 'INT[]'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv('00_data/sql/users.csv', header=True, columns={'user_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE couriers AS SELECT * FROM read_csv('00_data/sql/couriers.csv', header=True, columns={'courier_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE products AS SELECT * FROM read_csv('00_data/sql/products.csv', header=True, columns={'product_id': 'INT', 'name': 'VARCHAR', 'price': 'DOUBLE'});

,Count
0,87


# Фільтрація даних

Фільтрація даних дозволяє включати в результуючий запит не всі рядки вихідної таблиці, а лише ті, що відповідають заздалегідь заданій умові.

## Базова фільтрація: `WHERE`

В SQL для фільтрації даних використовується оператор `WHERE`. Після оператора `WHERE` вказується логічний вираз, результат якого визначає, чи буде рядок включений до результуючої таблиці. Якщо умова виявляється істинною (`TRUE`), то рядок включається до результату, якщо хибним (`FALSE`) — рядок виключається.

Таким чином, кожен рядок у таблиці проходить перевірку на відповідність певній умові, і в результаті цих перевірок формується таблиця, над якою проводяться операції, зазначені в блоці `SELECT`.

Оператор `WHERE` та логічний вираз вказуються після блоку `FROM`:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_2 >= 0
```

Наприклад, у результаті зазначеного вище запиту у вибірці попадуть лише записи з невід'ємними значеннями в колонці `column_2`.

У свою чергу оператори `ORDER BY` і `LIMIT` записуються вже після оператора `WHERE`. Якщо додати їх у приклад вище, запит виглядатиме так:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_2 >= 0
ORDER BY column_1
LIMIT 100
```

У результаті ми спочатку відфільтруємо необхідні нам рядки, потім виберемо стовпчики, виділені в `SELECT`, а потім сортуємо результуючу таблицю, обмеживши число виведених записів.

Таким чином порядок запису відомих нам на поточний момент ключових слів виглядає так:

1. `SELECT`
2. `FROM`
3. `WHERE`
4. `ORDER BY`
5. `LIMIT`

Знову звернемо увагу, що порядок виконання відрізняється від того, в якій послідовності вони вказуються в запиті:

1. Спочатку виконується оператор `FROM` – відбувається вибір потрібної таблиці.
2. Далі `WHERE` - відфільтровуються рядки, що відповідають умові.
3. Потім `SELECT` - відбираються зазначені стовпці та застосовуються функції.
4. Потім `ORDER BY` - проводиться сортування результуючої таблиці.
5. І наприкінці `LIMIT` — обмежується кількість записів, що виводяться.

Іншими словами, в результаті виконання запиту спочатку відбувається підготовка таблиці до роботи, а потім над нею виконуються різні операції.

::: {.callout-note}
Докладніше про оператор `WHERE` можна за [посиланням](https://duckdb.org/docs/sql/query_syntax/where).
:::

::: {#exr-sql-filter-01}
Напишіть SQL-запит до таблиці `products` та виведіть всю інформацію про товари, ціна яких не перевищує 100 одиниць. Результат відсортуйте за зростанням id товару.

Поля у результуючій таблиці: `product_id`, `name`, `price`

**Рішення:**
:::

In [2]:
%%sql
SELECT 
    product_id,
    name,
    price
FROM   products
WHERE  price <= 100
ORDER BY product_id
LIMIT  10;

,product_id,name,price
0,2,green tea bags,50.0
1,3,still water,80.0
2,4,lollipops,46.0
3,5,coffee 3 in 1,15.0
4,6,crackers,25.0
5,8,drying,30.0
6,9,black leaf tea,84.0
7,10,seeds,12.0
8,14,mayonnaise,60.0
9,18,ketchup,58.0


Фільтрувати дані в таблицях можна не тільки по полях з числовими значеннями, але і по полях зі значеннями, представленими у вигляді тексту:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_2 = 'text'
```

У прикладі вище в результуючу таблицю потраплять лише рядки, значення в яких повністю збігаються із зазначеним у `WHERE` рядком `'text'`.

При порівнянні рядків також допускається використовувати нерівності:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_2 > 'text'
```

Втім, така операція використовується рідше, оскільки не цілком очевидно, що означає «один рядок більше за інший».

Насправді порядок сортування даних рядкового типу зазвичай визначається заздалегідь встановленими правилами сортування, у яких значення мають довжина рядка в символах, порядок букв відповідно до алфавіту, наявність особливих символів, регістр тощо. Ми не будемо докладно зупинятись на цій темі — за бажання про це можна додатково прочитати у документації.

::: {.callout-note}
Докладніше про правила сортування даних рядкового типу можна за [посиланням](https://duckdb.org/docs/sql/expressions/collations).
:::

::: {#exr-sql-filter-02}
Виберіть користувачів жіночої статі з таблиці `users`. Виведіть лише id цих користувачів. Результат відсортуйте за зростанням id.

Додайте в запит оператор `LIMIT` і виведіть лише `10` перших ID з відсортованого списку.

Поле у результуючій таблиці: `user_id`

**Рішення:**

:::

In [3]:
%%sql
SELECT 
    user_id
FROM   
    users
WHERE  
    sex = 'female'
ORDER BY 
    user_id 
LIMIT 10

,user_id
0,1
1,2
2,3
3,4
4,5
5,7
6,8
7,11
8,12
9,13


Як значення для фільтрації можна також використовувати дати та час:

```{.sql}
SELECT column_1
FROM table
WHERE column_2 <= '2022-12-31'


SELECT column_1
FROM table
WHERE column_2 <= '2022-12-31 00:00:00'
```

Однак при цьому важливо заздалегідь переконатися, що в колонці, за якою фільтруватимуться дані, знаходяться саме дати або мітки часу, а не дані рядкового типу, зовні схожі на дати. Якщо в колонці виявляться рядки, то як такої помилки не відбудеться, проте результат сильно відрізнятиметься від очікуваного.

Також в операціях порівняння дат важливо враховувати, що дата завжди інтерпретується опівночі (початок дня), тобто `'2022-12-31'` насправді означає `'2022-12-31 00:00:00'`.

До речі, ніщо не забороняє нам комбінувати в блоці `WHERE` різні умови з різними типами даних і створювати складніші логічні вирази:


```{.sql}
SELECT column_1, column_2, column_3
FROM table
WHERE column_1 >= 0 
      AND column_2 = 'some text' 
      AND column_3 = '2022-12-31'
```

Давайте якраз вирішимо завдання одразу з кількома умовами!

::: {#exr-sql-filter-03}
Виберіть з таблиці `user_actions` усі дії користувачів зі створення замовлень, які були зроблені ними `після опівночі 6 вересня 2022 року`. Виведіть колонки з id користувачів, id створених замовлень та часом їх створення.

Результат має бути відсортований за зростанням id замовлення.

Поля в результуючій таблиці: `user_id`, `order_id`, `time`

**Пояснення:**

Зверніть увагу, що в таблиці user_actions у кожного користувача можуть бути записи не тільки з часом створення замовлення, а й часом його скасування. Нам необхідно отримати лише записи із створенням замовлень.

**Рішення:**
:::

In [4]:
%%sql
SELECT user_id,
       order_id,
       time
FROM   user_actions
WHERE  action = 'create_order' and time > '2022-09-06'
ORDER BY order_id

,user_id,order_id,time
0,16721,45201,2022-09-06 00:01:00
1,17299,45202,2022-09-06 00:01:00
2,7957,45203,2022-09-06 00:01:00
3,17300,45204,2022-09-06 00:02:00
4,14723,45205,2022-09-06 00:03:00
...,...,...,...
14390,21402,59591,2022-09-08 23:58:00
14391,20859,59592,2022-09-08 23:58:00
14392,15131,59593,2022-09-08 23:59:00
14393,10881,59594,2022-09-08 23:59:00


В операторі `WHERE` можна використовувати не тільки колонки, які вже є в таблиці, але й розрахункові колонки. При цьому вказувати їх у `SELECT` знову ж таки не обов'язково:


```{.sql}
SELECT column_1, column_2, (column_1 + column_2) * 0.5 AS average
FROM table
WHERE (column_1 + column_2) * 0.5 = 10


SELECT column_1, column_2
FROM table
WHERE (column_1 + column_2) * 0.5 = 10 
```

У прикладі вище ми відфільтруємо тільки ті записи, для яких середнє арифметичне двох чисел, що містяться в колонках `column_1` і `column_2`, дорівнюватиме `10`.

Також важливо враховувати, що використовувати в блоці `WHERE` аліаси, які присвоєно новим розрахунковим колонкам, не можна. Зверніть увагу на приклади наступних запитів, які вважають частку значень колонки `column_1` у відповідних їм значеннях колонки `column_2`. Перший запит спрацює, а другий поверне помилку:


```{sql}
-- Цей запит спрацює:

SELECT column_1, column_2, 
       column_1 / column_2 AS share
FROM table
WHERE column_1 / column_2 > 50


-- Цей запит поверне помилку:

SELECT column_1, column_2, 
       column_1 / column_2 AS share
FROM table
WHERE share > 50
```

Це пов'язано всі з тією ж послідовністю виконання операторів у запиті: на момент виконання оператора `WHERE` аліас колонці ще не присвоєно, і тому база даних поки не знає про існування колонки з новим ім'ям.

Можна розглянути ще більш тривіальний приклад: навіть якщо ми просто зробимо дублікат колонки з іншим ім'ям і спробуємо використати це ім'я в `WHERE`, все одно отримаємо помилку:


```{.sql}
-- Цей запит поверне помилку:

SELECT column, column AS same_column
FROM table
WHERE same_column > 0
```

При вказівці фільтрів у блоці `WHERE` важливо враховувати розглянуті особливості. Можете керуватися наступним правилом: якщо збираєтеся фільтрувати дані по розрахунковій колонці, то дублюйте розрахунки `WHERE` і не використовуйте аліаси, присвоєні новим колонкам.

::: {#exr-sql-filter-04}
Призначте знижку `20%` на всі товари з таблиці `products` та відберіть ті, ціна на які `з урахуванням знижки` перевищує `100`. Виведіть id товарів, їх найменування, колишню ціну та нову ціну з урахуванням знижки. Колонку зі старою ціною назвіть `old_price`, новою — `new_price`.

Результат має бути відсортований за зростанням id товару.

Поля в результуючій таблиці: `product_id`, `name`, `old_price`, `new_price`

**Пояснення:**

Будьте уважні, коли вказуватимете фільтр після оператора `WHERE`.

In [5]:
%%sql
SELECT product_id,
       name,
       price as old_price,
       price * 0.8 as new_price
FROM   products
WHERE  price * 0.8 > 100
ORDER BY product_id
LIMIT 10 -- для скорочення виводу

,product_id,name,old_price,new_price
0,1,sugar,150.0,120.0
1,7,grain coffee,351.0,280.8
2,11,linseed oil,200.0,160.0
3,12,grape,278.0,222.4
4,13,caviar,800.0,640.0
5,15,olive oil,450.0,360.0
6,17,сowberry juice,190.0,152.0
7,19,salted fish,180.0,144.0
8,21,ground coffee,283.0,226.4
9,25,instant coffee,150.0,120.0


Після оператора `WHERE` до колонок можна також застосовувати різні функції:

```{.sql}
SELECT column_1, column_2
FROM table 
WHERE LOWER(column_1) = 'i.love.python'
```

У прикладі вище кожне значення колонки `column_1` спочатку буде приведено до нижнього регістру, а потім зіставлено зі значенням 'i.love.python'. При цьому результат буде включено вихідні значення в колонці `column_1`. Таким чином, функція послужить тільки для фільтрації записів, але самі значення в колонках ніяк не вплине.

Зверніть увагу, що використовувати аліаси, присвоєні колонкам в операторі `SELECT`, так само не можна. Наступний запит поверне помилку:

```{.sql}
-- Цей запит поверне помилку:

SELECT column_1 AS new_column_1, column_2
FROM table 
WHERE LOWER(new_column_1) = 'i.love.python'
```

::: {#exr-sql-filter-05}
Відберіть із таблиці `products` всі товари, назви яких або починаються зі слова «`tea`», або складаються `з п'яти символів`. Виведіть дві колонки: id товарів та їх найменування.

Результат має бути відсортований за зростанням id товару.

Поля у результуючій таблиці: `product_id`, `name`

**Пояснення:**

Для вирішення завдання вам знадобляться функції `SPLIT_PART` та `LENGTH`, які ми розглядали раніше.

::: {.callout-tip collapse="true"}
## Підказка

Для вирішення завдання в операторі WHERE до колонки з назвами товарів необхідно застосувати функцію `SPLIT_PART`, розділивши по пробілу на окремі слова і відібравши тільки ті, в яких перше слово - «`tea`». Також через `OR` необхідно додати другу умову з функцією `LENGTH` для перевірки, що найменування складається з 5 символів.
:::

:::

In [6]:
%%sql
SELECT product_id,
       name
FROM   products
WHERE  split_part(name, ' ', 1) = 'tea'
    or length(name) = 5
ORDER BY product_id

,product_id,name
0,1,sugar
1,10,seeds
2,12,grape
3,26,chips
4,28,cream
5,34,pasta
6,36,tea mushroom
7,40,bread
8,52,kvass
9,53,flour


## Пошук за шаблоном: `LIKE`

Уявіть, що із загального списку найменувань товарів нам раптом знадобилося відібрати певні позиції, які містять певні слова і навіть символи. Але при цьому ми точно не знаємо, в якій частині назви їх можна зустріти.

Наприклад, нам потрібні всі "`tea`", але вони необов'язково повинні починатися зі слова "`tea`".

Але як це зробити? Невже нам доведеться переглядати всю таблицю, вручну складати список товарів і писати довгий логічний вираз, що враховує всі можливі сценарії?

Зрозуміло, що ні. Для фільтрації по колонках з текстовими значеннями в `SQL` передбачений оператор `LIKE`.

Оператор `LIKE` не просто порівнює рядки на повний збіг (або розбіжність), а перевіряє їх на відповідність заданому шаблону: якщо рядок йому відповідає, то повертається `TRUE`, інакше — `FALSE`. Як можна здогадатися, конструкція `NOT LIKE` працює з точністю навпаки.

Шаблони можуть містити як звичайні символи, так і символи-шаблони: знак відсотка (`%`) та підкреслення (`_`). Підкреслення підміняє будь-який одиночний символ, а знак відсотка - будь-яку (у тому числі порожню) послідовність символів:

```{.sql}
SELECT 'love.python' LIKE 'love%'

Результат:
true

SELECT 'love.python' LIKE 'love_'

Результат:
false

SELECT 'love.python' LIKE '%love%'

Результат:
true

SELECT 'love.python' LIKE '_love%'

Результат:
false

SELECT 'love.python' LIKE '%.%'

Результат:
true

SELECT 'love.python' LIKE '_._'

Результат:
false

SELECT 'love.python' LIKE 'Love%'

Результат:
false
```

Зверніть увагу на останній приклад: оператор `LIKE` чутливий до регістру.

Якщо шаблон не містить знаків відсотка та підкреслень, тоді шаблон є рядком і `LIKE` працює як оператор порівняння, перевіряючи рядки на точний збіг.

```{.sql}
SELECT 'love.python' LIKE 'love.python'

Результат:
true

SELECT 'love.python' LIKE 'love'

Результат:
false
```

::: {.callout-note}
Докладніше про оператор `LIKE` та шаблони можна почитати [тут](https://duckdb.org/docs/sql/functions/patternmatching.html).
:::

::: {#exr-sql-filter-06}
Відберіть із таблиці `products` всі товари, що містять у своїй назві послідовність символів «`tea`» (без лапок). Виведіть дві колонки: id продукту та його назву.

Результат має бути відсортований за зростанням id товару.

Поля у результуючій таблиці: `product_id`, `name`

**Рішення**
:::

In [7]:
%%sql
SELECT  product_id,
        name
FROM    products
WHERE   name LIKE '%tea%'
ORDER BY product_id

,product_id,name
0,2,green tea bags
1,9,black leaf tea
2,27,herbal leaf tea
3,36,tea mushroom
4,45,leaf green tea
5,48,ivan-tea in bags
6,50,cold tea
7,76,black tea bags
8,79,herbal tea bags


Тепер зробимо наш фільтр трохи цікавішим.

::: {#exr-sql-filter-07}
Виберіть із таблиці `products` id та найменування лише тих товарів, назви яких починаються на букву "`c`" і містять `лише одне слово`.

Результат має бути відсортований за зростанням id товару.

Поля у результуючій таблиці: `product_id`, `name`

::: {.callout-tip collapse="true"}
## Підказка

Якщо з буквою «`c`» все зрозуміло, то з фільтром на одне слово ситуація трохи складніша: подумайте, яким символом принципово відрізняються рядки з одним і двома словами. Складіть шаблон для пошуку рядка, що складається з двох слів і виключіть такі рядки з результату.

Також будьте уважні мовою введення: латинська літера "`c`" та кирилична "`с`" виглядають однаково, але для бази даних це різні символи.
:::

**Рішення**
:::

In [8]:
%%sql
SELECT product_id,
       name
FROM   products
WHERE  name like 'c%'
   and name not like '% %'
ORDER BY product_id

,product_id,name
0,6,crackers
1,13,caviar
2,26,chips
3,28,cream
4,46,cookie
5,73,cake
6,77,chicken
7,78,cupcakes


Уявіть, що напередодні Нового року до вас звернувся менеджер, якому на думку спала ідея зробити знижку на найдорожчий чай. Він попросив вас надати йому вивантаження з бази даних з усіма чаями та новими цінами на них з урахуванням знижки.

Що він далі робитиме з цими даними, вам невідомо, але завдання є завдання, до того ж не таке вже й складне. Спробуймо його вирішити!

::: {#exr-sql-filter-08}
Складіть SQL-запит, який вибирає з таблиці `products` **всі чаї вартістю більше 60** і нараховує їм **знижку 25%**.

Знижку в % менеджер попросив вказати в окремому стовпці у форматі тексту, тобто так: «**25%**» (без лапок). Стовпці зі знижкою та новою ціною назвіть відповідно `discount` та `new_price`.

Також необхідно будь-яким відомим способом позбавитися «**tea mushroom**»: навряд чи менеджер мав на увазі і його, коли ставив нам завдання.

Результат має бути відсортований за зростанням id товару.

Поля в результуючій таблиці: `product_id`, `name`, `price`, `discount`, `new_price`

**Пояснення:**

Щоб вказати стовпець з одним значенням для всіх записів (у нашому випадку це стовпець зі знижкою), достатньо просто ввести потрібне значення і дати стовпцю якусь назву - вказане значення автоматично проставиться у всіх рядках результуючої таблиці:

```{.sql}
SELECT column_1, 'text' AS column_2
FROM table
```

**Рішення:**
:::

In [9]:
%%sql
SELECT product_id,
       name,
       price,
       '25%' as discount,
       price * 0.75 as new_price
FROM   products
WHERE  price > 60
   and name != 'tea mushroom'
   and name like '%tea%'
ORDER BY product_id

,product_id,name,price,discount,new_price
0,9,black leaf tea,84.0,25%,63.00
1,27,herbal leaf tea,121.0,25%,90.75
2,45,leaf green tea,78.0,25%,58.50
3,79,herbal tea bags,65.0,25%,48.75


## `IN` та `BETWEEN` для фільтрації

Але якщо ми все-таки не хочемо перевіряти наші дані на відповідність якомусь шаблону, а просто хочемо відібрати значення з деякого списку або навіть діапазону? У цьому випадку у логічному виразі після ключового слова WHERE можна використовувати оператори `IN` та `BETWEEN`.

Оператор `IN` перевіряє, чи відповідає значення в колонці одному із значень із заданого списку. Іншими словами, він перевіряє, чи входить значення до цього списку:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_1 IN ('product_1', 'product_2', 'product_3')
```

У свою чергу, оператор `BETWEEN` дозволяє відібрати дані, що відносяться до деякого інтервалу. При цьому межі інтервалу включаються:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_2 BETWEEN 5 AND 10
```

Запит вище відповідає наступному запиту:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_2 >= 5 AND column_2 <= 10
```

Оператор `BETWEEN` можна використовувати і для фільтрації по колонках з датами та часом:

```{.sql}
SELECT column_1, column_2, column_3
FROM table
WHERE column_3 BETWEEN '2022-11-20' AND '2022-12-31'
```

Однак, в такому випадку, треба враховувати, що дата завжди інтерпретується опівночі (початок дня), тобто. **'2022-12-31'** насправді означає **'2022-12-31 00:00:00'**. Тож заданий вище інтервал не потраплять записи пізніше опівночі **'2022-12-31'**, тобто цей день практично не буде враховано в інтервалі.

При цьому, запит вище буде рівносильний наступному запиту:

```{.sql}
SELECT column_1, column_2, column_3
FROM table
WHERE column_3 >= '2022-11-20' AND column_3 <= '2022-12-31'
```

Для отримання зворотного результату у поєднанні з операторами `IN` та `BETWEEN` можна використовувати оператор `NOT`:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_1 NOT IN ('product_1', 'product_2', 'product_3')


SELECT column_1, column_2
FROM table
WHERE column_2 NOT BETWEEN 5 AND 10
```

::: {.callout-note}
Детальніше про оператори `IN` та `BETWEEN` можна прочитати [тут](https://duckdb.org/docs/sql/expressions/in.html) і [тут](https://duckdb.org/docs/sql/expressions/comparison_operators.html).
:::

::: {#exr-sql-filter-09}
З таблиці `user_actions` виведіть всю інформацію про дії користувачів з id **170**, **200** та **230** за період **з 25 серпня до 4 вересня 2022 року включно**. Результат відсортуйте за зменшенням id замовлення - тобто від останніх до найпізніших.

Поля у результуючій таблиці: `user_id`, `order_id`, `action`, `time`

**Рішення:**
:::

In [10]:
%%sql
SELECT user_id,
       order_id,
       action,
       time
FROM   user_actions
WHERE  user_id in (170, 200, 230)
   and time >= '2022-08-25'
   and time < '2022-09-05' -- уважно з датами!
ORDER BY order_id desc

,user_id,order_id,action,time
0,170,34428,create_order,2022-09-04 01:43:00
1,170,34424,create_order,2022-09-04 01:42:00
2,230,24675,create_order,2022-09-02 12:21:00
3,200,24538,create_order,2022-09-02 11:50:00
4,230,22887,create_order,2022-09-02 00:54:00
5,230,22878,create_order,2022-09-02 00:50:00
6,230,19543,create_order,2022-09-01 07:21:00
7,170,14894,create_order,2022-08-30 22:04:00
8,230,14466,create_order,2022-08-30 19:51:00
9,170,4194,create_order,2022-08-27 17:28:00


Раніше @exr-sql11 ми зіткнулися з пропущеними значеннями в таблиці `couriers` — у деяких кур'єрів не було зазначено їхні дні народження.

Для перевірки на `NULL` значення **SQL** є оператор `IS NULL`. У поєднанні з `WHERE` записується він так:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_1 IS NULL
```

Якщо ж, навпаки, необхідно відібрати значення `не NULL`, то додатково використовується оператор `NOT`:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_1 IS NOT NULL
```

Як ми знаємо, у світі даних `NULL` означає відсутність інформації. `NULL` - це не якась величина і тому її не можна порівняти з чимось ще. Результатом порівняння `NULL` з будь-яким іншим значенням буде той самий `NULL`. Більше того, `NULL` не вдасться порівняти і з іншим `NULL`, тому що в такому випадку порівнюватимуться дві невизначеності і не можна напевно сказати, рівні вони чи ні.

Спробуйте виконати наступні запити:

```{.sql}
SELECT NULL = NULL

Результат:
NULL

SELECT NULL IS NULL

Результат:
true

SELECT 100 = NULL

Результат:
NULL

SELECT 100 IS NULL

Результат:
false
```

::: {.callout-note}
Детальніше про значення `NULL` можна прочитати [тут](https://duckdb.org/docs/sql/data_types/nulls.html).
:::